In [2]:
import pandas as pd
import random

df = pd.read_excel('/home/fantoni/patent-sentence-classification/data/6000_axiomatic_dataset.xlsx')

In [3]:
def random_split(df, train_ratio=0.7, eval_ratio=0.2, seed=1999):
    random.seed(seed)
    # Shuffle the entire DataFramme
    df = df.sample(frac=1, random_state=seed).reset_index(drop=True)
    
    # Calculate split indices
    train_end = int(len(df) * train_ratio)
    validation_end = train_end + int(len(df) * eval_ratio)
    
    # Split the DataFrame
    train_set = df[:train_end]
    eval_set = df[train_end:validation_end]
    test_set = df[validation_end:]
    
    return train_set, eval_set, test_set

train_set, eval_set, test_set = random_split(df)
train_set.to_excel(f"/home/fantoni/patent-sentence-classification/data/train.xlsx", index=False)
eval_set.to_excel(f"/home/fantoni/patent-sentence-classification/data/eval.xlsx", index=False)
test_set.to_excel(f"/home/fantoni/patent-sentence-classification/data/test.xlsx", index=False)
print("Data successfully split and saved.")

Data successfully split and saved.
